In [ ]:
import warnings
warnings.filterwarnings("ignore")
import os
import contextlib
with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
    from manim import *
import autograd.numpy as np


class LectureScene(Scene):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.camera.background_color = "#ffffff"
        self.template = TexTemplate()
        self.template.add_to_preamble(r"\usepackage{amsmath}")

class ThreeDLectureScene(ThreeDScene):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.camera.background_color = "#ffffff"
        self.template = TexTemplate()
        self.template.add_to_preamble(r"\usepackage{amsmath}")
    

class VectorScene(LectureScene):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.ax = Axes(
            x_range=[-7.5, 7.5, 1],
            y_range=[-5, 5, 1],
            x_length=12,
            y_length=8,
            axis_config={"color": GREY},
        )
        
        #axes_labels.set_color(GREY)
        self.add(self.ax)

class PositiveVectorScene(LectureScene):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.ax = Axes(
            x_range=[-2.5, 12.5, 1],
            y_range=[-1, 9, 1],
            x_length=12,
            y_length=8,
            axis_config={"color": GREY},
        )
                #axes_labels.set_color(GREY)
        self.add(self.ax)

class ComparisonVectorScene(LectureScene):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.ax1 = Axes(
            x_range=[-5, 5, 1],
            y_range=[-5, 5, 1],
            x_length=6,
            y_length=6,
            axis_config={"color": GREY},
        )
        self.ax2 = Axes(
            x_range=[-5, 5, 1],
            y_range=[-5, 5, 1],
            x_length=6,
            y_length=6,
            axis_config={"color": GREY},
        )
        axgroup = Group(self.ax1, self.ax2)
        axgroup.arrange_in_grid(buf=2)
        
        #axes_labels.set_color(GREY)
        self.add(axgroup)
from manim_ml.neural_network import NeuralNetwork, Convolutional2DLayer, MaxPooling2DLayer

In [ ]:
type(BLACK)

In [ ]:
%%manim -pql -v WARNING Conv
def make_sequence(numbers, sl=0.5, colors=BLACK, fills=WHITE, text_colors=None, opacity=1.,  annot=True, stride=1, padding=0):
    n = len(numbers)
    squares = []

    if type(colors) is str:
        colors = [colors] * n
    if text_colors is None:
        text_colors = colors
    elif type(text_colors) is str:
        text_colors = [text_colors] * n
    if type(fills) is str:
        fills = [fills] * n

    for i in range(n):
        pos = i * stride * sl * RIGHT
        square = Square(color=colors[i], fill_color=fills[i], side_length=sl,  fill_opacity=opacity).move_to(pos)
        if annot:
            text = Text(str(numbers[i]), color=text_colors[i]).scale(sl).move_to(pos)
            squares.append(VGroup(square, text))
        else:
            squares.append(VGroup(square))
    
    if padding > 0:
        pads = []
        for i in range(padding):
            pos = (n + i) * sl * RIGHT
            pads.append(Text('0', color=GRAY).scale(sl).move_to(pos))
            pos = (-(i + 1)) * sl * RIGHT
            pads.append(Text('0', color=GRAY).scale(sl).move_to(pos))
        return VGroup(*squares), VGroup(*pads)

    return VGroup(*squares)

def make_stack(numbers, sl=0.5, colors=BLACK, fills=WHITE, text_colors=None, opacity=1., annot=True, stride=1):
    n = len(numbers)

    if type(colors) is str:
        colors = [colors] * n
    if text_colors is None:
        text_colors = colors
    elif type(text_colors) is str:
        text_colors = [text_colors] * n
    if type(fills) is str:
        fills = [fills] * n

    seqs = []
    for i in range(n):
        seqs.append(make_sequence(numbers[i], sl, colors[i], fills[i], text_colors[i], opacity, annot, stride).shift(i * sl * DOWN))
    return VGroup(*seqs)

class Conv(LectureScene):
    def construct(self):
        self.add(Text('Input', color=GRAY, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + -2 * UP))
        self.add(Text('Kernel', color=RED, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT + -1 * UP))
        self.add(Text('Output', color=GRAY, opacity=0.5, font='Noto Sans').scale(0.6).move_to(5 * LEFT))
        self.add(Text('Convolution (kernel size: 3)', color=BLACK,  font='Noto Sans SemiBold').scale(0.8).move_to(3.5 * UP))


        kernelarr = [-1, 3, -1]
        kw = len(kernelarr) // 2
        kernel = make_sequence(kernelarr, annot=True, opacity=0.2, fills=RED, colors=RED).move_to(-1 * UP)
        seqarr = np.random.randint(-9, 9, (15,))

        sequence = make_sequence(seqarr, annot=True).move_to(-2 * UP)
        outputarr = [sum([seqarr[s - kw + i] * k for i, k in enumerate(kernelarr)]) for s in range(kw, len(seqarr) - kw)]

        output = make_sequence(outputarr, annot=True, text_colors=WHITE).move_to(0 * UP)

        output_window = make_sequence([1], annot=False, opacity=.2, fills=RED, colors=RED).move_to(0 * UP)
        kernel_window = make_sequence([1, 1, 1], annot=False, fills=RED, opacity=0.2, colors=RED).move_to(-2 * UP)

        lines = []
        for s in [-0.5, 0., 0.5]:
            l = Line(-1.75 * UP, -1.25 * UP, color=RED).shift(s * RIGHT)
            lab = MathTex(r'\times', color=RED).scale(0.5).next_to(l, 0.5 * LEFT) 
            l2 = CubicBezier(-.75 * UP + s * RIGHT, -.5 * UP + s * RIGHT, -.5 * UP, -.25 * UP, color=RED)
            lines += [l, lab, l2]
        lines.append(MathTex(r'+', color=RED).scale(0.5).next_to(l2, 1.5 * LEFT).shift(0.1 * UP))
        lines.append(kernel_window)
        kernel_window = VGroup(*lines)

        highlight = VGroup(output_window, kernel_window, kernel)
        highlight.shift(0.5 * 6 * LEFT)
        eq = Text(f'Output[{0 :2}]=({seqarr[0] :2})({kernelarr[0] :2})+({seqarr[1] :2})({kernelarr[1] :2})+({seqarr[2] :2})({kernelarr[2] :2})={outputarr[0] :3}', color=BLACK, font_size=36, font='Roboto Mono').shift(2 * UP)
        self.add(sequence, output, highlight, eq)

                
        output[0][1].set_color(BLACK)
        for i in range(12):
            neweq = Text(f'Output[{i+1 :2}]=({seqarr[0 + i] :2})({kernelarr[0] :2})+({seqarr[1 + i] :2})({kernelarr[1] :2})+({seqarr[2 + i] :2})({kernelarr[2] :2})={outputarr[1 + i] :3}', color=BLACK, font_size=36, font='Roboto Mono').shift(2 * UP)
            self.play(highlight.animate.shift(RIGHT * 0.5), output[i + 1][1].animate.set_color(BLACK), FadeIn(neweq), FadeOut(eq))
            eq = neweq
        self.wait(2)


In [ ]:
class Mat

In [ ]:
%%manim -pql -v WARNING MovingAround

class MovingAround(LectureScene):
    def construct(self):
        sentence = "[START] I love purple cats [.] Purple cats are great [END]".split()
        length = len(sentence)
        heights = np.linspace(1, -1, length)
        yrange = 2
        rowheight = (heights[1] - heights[0]) * yrange

        dim = 3

        embeddings = np.random.randn(length, dim)
        rows = []
        for r, (word, y) in enumerate(zip(sentence, heights)):
            label = Text(word, color='BLACK').scale( 2 * yrange * (1 / length)).shift(-5 * RIGHT + yrange * y * UP)

            row = []
            for c in range(dim):
                box = Square(side_length=rowheight, color=BLACK, fill_color=LIGHTER_GRAY, fill_opacity=1.).shift(-3 * RIGHT + yrange * y * UP).shift(rowheight * RIGHT * c)
                num = Text('%.2f' % embeddings[r, c], color=BLACK).scale( 1.25 * yrange * (1 / length) ).shift(-3 * RIGHT + yrange * y * UP).shift(rowheight * RIGHT * c)
                row.append(VGroup(box, num))
            rows.append(VGroup(label, VGroup(*row)))
        rows = VGroup(*rows)

        wq = np.random.randn(dim, dim)
        wk = np.random.randn(dim, dim)
        wv = np.random.randn(dim, dim)

        ms = []
        for (mat, y, color) in [(wq, 2, GREEN), (wk, 0, BLUE), (wv, -2, ORANGE)]:
            m = []
            for r in range(dim):
                for c in range(dim):
                    box = Square(side_length=rowheight, color=BLACK, fill_color=LIGHTER_GRAY, fill_opacity=1.).shift(-1 * RIGHT +  y * UP).shift(rowheight * RIGHT * c + rowheight * UP * r)
                    num = Text('%.2f' % mat[r, c], color=BLACK).scale( 1.25 * yrange * (1 / length) ).shift(-1 * RIGHT + y * UP).shift(rowheight * RIGHT * c + rowheight * UP * r)
                    m.append(VGroup(box, num))
            ms.append(VGroup(*m))
        ms = VGroup(*ms)

        self.add(rows, ms)

In [ ]:
def convolution1d(x, kernel):
    kernel_width = len(kernel)
    input_size = len(x)

    output_size = input_size - (kernel_width - 1)
    output = np.zeros((output_size,))
    for i in range(output_size):
        for j in range(kernel_width):
            output[i] += x[i + j] * kernel[j]
    return output

In [ ]:
def convolution1d(x, kernel):
    kernel_width = len(kernel)
    input_size = len(x)

    output_size = input_size - (kernel_width - 1)
    output = np.zeros((output_size,))
    for i in range(output_size):
        for j in range(kernel_width):
            output[i] += np.dot(x[i + j], kernel[j])
    return output

In [ ]:
def convolution1d(x, kernel, padding=0):

    kernel_width = len(kernel)
    input_size = len(x)

    output_size = input_size - (kernel_width - 1) + 2 * padding
    output = np.zeros((output_size,))
    for i in range(output_size):
        for j in range(kernel_width):
            ind = i + j - padding
            if ind >= 0 and ind < input_size:
                output[i] += np.dot(x[ind], kernel[j])
    return output

In [ ]:
def convolution1d(x, kernel, padding=0, stride=1):

    kernel_width = len(kernel)
    input_size = len(x)

    output_size = (input_size - (kernel_width - 1) + 2 * padding - 1) // stride + 1
    output = np.zeros((output_size,))
    for i in range(len(output)):
        for j in range(kernel_width):
            ind = i * stride + j - padding
            if ind >= 0 and ind < input_size:
                output[i] += np.dot(x[ind], kernel[j])
    return output

In [ ]:
def convolution1d(x, kernel, padding=0, stride=1, dialation=1):

    kernel_width = len(kernel)
    input_size = len(x)

    output_size = (input_size - dialation * (kernel_width - 1) + 2 * padding - 1) // stride + 1
    output = np.zeros((output_size,))
    for i in range(len(output)):
        for j in range(kernel_width):
            ind = i * stride + j * dialation - padding
            if ind >= 0 and ind < input_size:
                output[i] += np.dot(x[ind], kernel[j])
    return output

In [ ]:
def convolution1d(x, kernel, bias=0, padding=0, stride=1, dialation=1):

    kernel_width = len(kernel)
    input_size = len(x)

    output_size = (input_size - dialation * (kernel_width - 1) + 2 * padding - 1) // stride + 1
    output = np.zeros((output_size,))
    for i in range(len(output)):
        output[i] = bias
        for j in range(kernel_width):
            ind = i * stride + j * dialation - padding
            if ind >= 0 and ind < input_size:
                output[i] += np.dot(x[ind], kernel[j])
    return output

In [ ]:
def maxpool1d(x, window, padding=0, stride=1):
    input_size = len(x)

    output_size = (input_size - (window - 1) + 2 * padding - 1) // stride + 1
    output = np.zeros((output_size,))
    for i in range(len(output)):
        output[i] = -np.infty
        for j in range(window):
            ind = i * stride + j - padding
            if ind >= 0 and ind < input_size:
                output[i] = np.maximum(x[ind], output[i])
    return output

In [ ]:
def avgpool1d(x, window, padding=0, stride=1):
    input_size = len(x)

    output_size = (input_size - (window - 1) + 2 * padding - 1) // stride + 1
    output = np.zeros((output_size,))
    for i in range(len(output)):
        output[i] = 0.
        for j in range(window):
            ind = i * stride + j - padding
            if ind >= 0 and ind < input_size:
                output[i] += x[ind] / window
    return output

In [ ]:

from manim import *

In [ ]:
%%manim -pql -v WARNING MovingAround

from manim_ml.neural_network import NeuralNetwork, FeedForwardLayer
class MovingAround(Scene):
    def construct(self):
        nn = NeuralNetwork([
            FeedForwardLayer(num_nodes=3),
            FeedForwardLayer(num_nodes=5),
            FeedForwardLayer(num_nodes=3)
        ])
        self.add(nn)
        # Make the animation
        forward_pass_animation = nn.make_forward_pass_animation()
        # Play the animation
        self.play(forward_pass_animation)

In [ ]:
import manim_ml

In [ ]:
manim_ml.ThreeDScene